In [1]:
# this enable automatic reloading of libraries
%load_ext autoreload
%autoreload 2

from clib.flow import *
from clib.bellhop import *
import subprocess
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
s = bellhop({'mediterrannee': '../SSP_4profils.mat'},name='profiles')

In [206]:
### read file .arr 
#Arr1, Pos1 = s.read_arrivals_asc ('files/med1.arr', Narrmx = 2785)

Arr1, Pos1 =   s.read_arrivals_asc ('tests/med1_same.arr', Narrmx = 2787)
#Arr1, Pos1 = s.load_dict('../Dictio', 'Arr1'), s.load_dict('../Dictio', 'Pos1')

Max.number of arrivals for source index 0 is 2786


In [207]:
### Reponses impulsionnelles
D = np.array([100., 200., 300., 400.])
R = np.array([10., 20., 30., 40., 50., 60., 70., 80., 90.]) * 1000.
print('Nombre total de réponses impulsionnelles : ', D.size * R.size)

### Indices correspondants aux récepteurs choisis
IRD = np.zeros_like(D)
for i in range (D.size) : 
    IRD[i] = np.where(Pos1['r']['depth']==D[i])[0][0]
    
IRR = np.zeros_like(R)
for i in range (R.size) : 
    IRR[i] = np.where(Pos1['r']['range']==R[i])[0][0]

Nombre total de réponses impulsionnelles :  36


# Méthode de détection : 1ere arrivée > 10 dB

In [208]:
# sonar equation
SL = 185                 # Souce level (dB)
NL = 81.7                # Noise level (dB)
DI = 0                   # Directivity index (dB)
B = 200                  # bandwidth (Hz)
T = 1                    # transmitted signal duration (s)
PG = 10.0*np.log10(B*T)  # Processing gain (dB)

# SL - TL - NL + DI + PG  >  RL 

In [218]:
### MAX D'AMPLITUDE

isd = 0       # choice of source depth
    
Tarr = []
TopBnc = []
BotBnc = []
SrcAngle = []


Tarr_Test = []

k = 0
for i in range (IRD.size):
    for j in range(IRR.size):
        k+=1

        ird = int(IRD[i])     # choice of receiver depth
        irr = int(IRR[j])     # choice of receiver range
        
        # Received level
        Narr = int(Arr1['Narr'][irr,ird,isd])
        amp = Arr1['A'][irr,:Narr, ird, isd]
        TL = -20*np.log10 (abs(amp))
        RL = SL - TL -NL +DI + PG
        
        # Times
        times = Arr1['delay'][irr, :Narr, ird, isd]
        idx_first = np.where(RL==np.max(RL))[0][0]
        
        # Chosen ray features
        NumTopBnc_f = Arr1['NumTopBnc'][irr, :Narr, ird, isd][idx_first]
        NumBotBnc_f = Arr1['NumBotBnc'][irr, :Narr, ird, isd][idx_first]
        SrcAngle_f  = Arr1['SrcAngle'][irr, :Narr, ird, isd][idx_first]

        # Appends
        TopBnc.append(NumTopBnc_f)
        BotBnc.append(NumBotBnc_f)
        SrcAngle.append(SrcAngle_f)
        Tarr.append(times[idx_first])

    
    
    
        ### test avec ancienne méthode

        t = Arr1['delay'][irr, :Narr, ird, isd]             # arrival time   
        idx_max = np.where(RL==np.max(RL))
        Tarr_Test.append(t [idx_max][0]) 
        

TimeA = np.array(Tarr)
Tarr_TestA = np.array(Tarr_Test)    

In [219]:
### SEUIL DE DETECTION A 10dB

isd = 0       # choice of source depth
seuil_d = 10.
    
Tarr = []
TopBnc = []
BotBnc = []
SrcAngle = []


Tarr_Test = []

k = 0
for i in range (IRD.size):
    for j in range(IRR.size):
        k+=1

        ird = int(IRD[i])     # choice of receiver depth
        irr = int(IRR[j])     # choice of receiver range
        
        # Received level
        Narr = int(Arr1['Narr'][irr,ird,isd])
        amp = Arr1['A'][irr,:Narr, ird, isd]
        TL = -20*np.log10 (abs(amp))
        RL = SL - TL -NL +DI + PG
        
        # Times
        times = Arr1['delay'][irr, :Narr, ird, isd]
        idx = np.where(RL > 10.)
        t_ok = times[idx]
        if len(t_ok) != 0 : 
            t_first = np.min(t_ok)
            idx_first = np.where(times==t_first)[0][0]
        
            # Chosen ray features
            NumTopBnc_f = Arr1['NumTopBnc'][irr, :Narr, ird, isd][idx_first]
            NumBotBnc_f = Arr1['NumBotBnc'][irr, :Narr, ird, isd][idx_first]
            SrcAngle_f  = Arr1['SrcAngle'][irr, :Narr, ird, isd][idx_first]
        
            # Appends
            TopBnc.append(NumTopBnc_f)
            BotBnc.append(NumBotBnc_f)
            SrcAngle.append(SrcAngle_f)
            Tarr.append(t_first)
            
        else : 
            TopBnc.append(np.NaN)
            BotBnc.append(np.NaN)
            SrcAngle.append(np.NaN)
            Tarr.append(np.NaN)
    
    
    
        ### test avec ancienne méthode

        t = Arr1['delay'][irr, :Narr, ird, isd]             # arrival time   
        t_ok = t[np.where(RL > seuil_d)]  
        if len(t_ok) > 0: 
            t_min = np.min(t_ok)
            idx_tresh = np.where(t==t_min)
            Tarr_Test.append(t[idx_tresh[0]])

        else : 
            Tarr_Test.append(np.NaN)


Time = np.array(Tarr)
Tarr_Test = np.array(Tarr_Test)    

In [217]:
Tarr_Test - Time

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [220]:
Tarr_TestA - TimeA

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [10]:
### On connaît le temps d'arrivée et le range correspondant pour les 20 vignettes
TT = np.array(Tarr)
REAL_RANGE = np.zeros(D.size*R.size)
for k in range (D.size) :
    REAL_RANGE[k*R.size:(k+1)*R.size]=R
    
PROF = np.zeros(D.size*R.size)
for k in range (D.size) :
    PROF[k*R.size:(k+1)*R.size] = D[k]

In [11]:
### 1 seule régression linéaire 
res = np.polyfit(TT, REAL_RANGE,1)
Rreg = res[0] * TT + res[1]
Diff1 = REAL_RANGE - Rreg

In [12]:
### autre méthode, qui nous donne le coef de détermination r²
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(TT, REAL_RANGE)

#To get coefficient of determination (r_squared)
print("r-squared:", r_value**2)

r-squared: 0.99998975946889


In [13]:
D_EQ = np.sqrt((REAL_RANGE - 0)**2 + (PROF - Pos1['s']['depth'][0])**2)

### Ancien calcul 

In [248]:
#Arr, Pos = s.read_arrivals_asc ('../files_arr/cognac_2000m_test_A_full.arr', Narrmx = 15)

#Arr, Pos = s.load_dict('../Dictio', 'Arr1'), s.load_dict('../Dictio', 'Pos1')
#Arr, Pos =   s.read_arrivals_asc ('tests/med1_OLD.arr', Narrmx = 7752)
#Arr, Pos =   s.read_arrivals_asc ('tests/med1_51_101.arr', Narrmx = 3883)
#Arr, Pos =   s.read_arrivals_asc ('tests/med1_12000NBeams.arr', Narrmx = 1963)
Arr, Pos =   s.read_arrivals_asc ('tests/med1_density_ok.arr', Narrmx = 7752)
#Arr, Pos =   s.read_arrivals_asc ('tests/med1_same.arr', Narrmx = 2787)

Max.number of arrivals for source index 0 is 1962


In [249]:

Nsd = np.shape(Pos['s']['depth'])[0]
Nrd = np.shape(Pos['r']['depth'])[0]
Nrr = np.shape(Pos['r']['range'])[0]
print('Number of source depths   : Nsd = ', Nsd)
print('Number of receiver depths : Nrd = ', Nrd)
print('Number of receiver ranges : Nrr = ', Nrr)
print('\nTotal number of receivers : ', Nrd*Nrr)

Number of source depths   : Nsd =  1
Number of receiver depths : Nrd =  6
Number of receiver ranges : Nrr =  11

Total number of receivers :  66


In [250]:
dstep = 1    #100      # depth step (here 100m)
rstep = 1    #1000     # range step (here 10km)

In [252]:
###SSP values
issp = 0
SSP_value = s.SSP['mediterrannee']['c'][issp,:]
depth = s.SSP['mediterrannee']['depth'][:]

###Sonar equation
seuil_d = 10.            # Detection treshold (dB)

###Variables
Dist_eq    = np.zeros( (int(Nrr/rstep)+1, int(Nrd/dstep)+1, Nsd) )    # distance source-receiver
#maximum amplitude
Tarr_A       = np.zeros( (int(Nrr/rstep)+1, int(Nrd/dstep)+1, Nsd) )    # time of arrival
#detection treshold
Tarr_T       = np.zeros( (int(Nrr/rstep)+1, int(Nrd/dstep)+1, Nsd) )    # time of arrival



for isd1 in range (Nsd):
    #for ird1 in range (0,Nrd+1,dstep) :
    for ird1 in range (Nrd) : 
        #for irr1 in range (0,Nrr+1,rstep) :
        for irr1 in range (Nrr) :
            Narr = int(Arr['Narr'][irr1, ird1, isd1])
            if not Narr ==0 :


                ## data from arrivals (A)
                numBot_A = Arr['NumBotBnc'][irr1, :Narr, ird1, isd1]  # bottom reflections
                numTop_A = Arr['NumTopBnc'][irr1, :Narr, ird1, isd1]  # top reflections
                t = Arr['delay'][irr1, :Narr, ird1, isd1]             # arrival time                  
                A = abs(Arr['A'][irr1, :Narr, ird1, isd1])            # amplitude
                TL = - 20.0*np.log10(A)                               # transmission losss 
                RL = SL - TL - NL +DI + PG                            # received level


                ################## Amplitude ######################

                idx_max = np.where(RL==np.max(RL))
                Tarr_A [int(irr1/rstep), int(ird1/dstep), isd1] = t [idx_max][0] 


                ################### Treshold ######################

                t_ok = t[np.where(RL > seuil_d)]                    
                if len(t_ok > 0): 
                    t_min = np.min(t_ok)
                    idx_tresh = np.where(t==t_min)[0][0]
                    Tarr_T [int(irr1/rstep), int(ird1/dstep), isd1] = t[idx_tresh]

                else : 
                    Tarr_T [int(irr1/rstep), int(ird1/dstep), isd1] = np.NaN
                    
                    

                ### test avec ancienne méthode
 
                #t_ok = t[np.where(RL > seuil_d)]  
                #if len(t_ok) > 0: 
                #    t_min = np.min(t_ok)
                #    idx_tresh = np.where(t==t_min)
                #    Tarr_Test.append(t[idx_tresh[0]])

                #else : 
                #    Tarr_Test.append(np.NaN)



                ############## Equivalent distance ################

                sr = 0.                           # source range(m)
                sd = Pos['s']['depth'][isd1]      # source depth (m)
                rr = Pos['r']['range'][irr1]      #receiver range (m)
                rd = Pos['r']['depth'][ird1]      #receiver depth (m)
                dist_eq = np.sqrt( (rr-sr)**2 + (rd-sd)**2)
                Dist_eq [int(irr1/rstep), int(ird1/dstep), isd1] = dist_eq

            
            else : 
                Tarr_A [int(irr1/rstep), int(ird1/dstep), isd1] = np.NaN
                Tarr_T [int(irr1/rstep), int(ird1/dstep), isd1] = np.NaN
                Dist_eq [int(irr1/rstep), int(ird1/dstep), isd1] = np.NaN


Dist_eq = Dist_eq[1:-2,1:-2].squeeze().T
#Dist_eq = Dist_eq[:-1,:-1].squeeze().T
#Dist_eq = Dist_eq.T.flatten()

Tarr_T = Tarr_T[1:-2,1:-2].squeeze().T
#Tarr_T = Tarr_T[:-1,:-1].squeeze().T
#Tarr_T = Tarr_T.T.flatten()


Tarr_A = Tarr_A[1:-2,1:-2].squeeze().T
#Tarr_A = Tarr_A[:-1,:-1].squeeze().T
#Tarr_A = Tarr_A.T.flatten()

In [198]:
D_EQ - Dist_eq.flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [169]:
Tarr_T

array([[ 6.62018299, 13.1910839 , 19.7801056 , 26.3735714 , 33.0494041 ,
        39.6289673 , 46.1730995 , 52.8712463 , 59.4673882 ],
       [ 6.61237144, 13.1860409 , 19.7811298 , 26.3672218 , 32.9545631 ,
        39.6200676 , 46.161869  , 52.8628426 , 59.4578819 ],
       [ 6.60918903, 13.179822  , 19.782156  , 26.3608742 , 32.9580612 ,
        39.6111679 , 46.1490021 , 52.8544426 , 59.4483757 ],
       [ 6.60591412, 13.1735964 , 19.7831821 , 26.3808079 , 32.9615593 ,
        39.601387  , 46.1476822 , 52.8460388 , 59.9470787 ]])

In [205]:
Time_OK = Time
Time_OK

array([ 6.62478209, 13.1912718 , 19.7972031 , 26.3745308 , 33.0499725 ,
       39.6280632 , 46.1828499 , 52.8760643 , 59.4247093 ,  6.62170219,
       13.1862659 , 19.7916851 , 26.3685207 , 33.0195961 , 39.6196098 ,
       46.1726761 , 53.1003685 , 59.4145126 ,  6.61241865, 13.1809559 ,
       19.7825336 , 26.3625088 , 32.9886513 , 39.612957  , 46.1610107 ,
       52.8917923 , 59.4312096 ,  6.60736609, 13.1747932 , 19.7842598 ,
       26.3564987 , 32.9847679 , 39.6043701 , 46.1503983 , 52.8253021 ,
       59.4267044 ])

In [211]:
Time - Time_OK

array([ 5.9318000e-04, -7.6000000e-06, -1.5946000e-03, -1.3905000e-03,
       -2.3109400e-02, -4.0440000e-04, -1.6491000e-02, -3.2257100e-02,
        1.5260000e-04, -2.7894900e-03, -1.7730000e-04, -1.0974900e-02,
       -5.7220000e-04,  9.1629000e-03, -1.5525000e-03, -1.8474600e-02,
       -2.6659010e-01,  4.4250000e-04, -1.2340500e-03, -1.2121000e-03,
       -3.8140000e-04,  1.3540000e-04,  3.6666800e-02, -4.9057000e-03,
       -1.0528500e-02, -6.5986600e-02,  1.0224876e+00,  4.6250000e-05,
       -1.3895000e-03, -1.0414000e-03,  2.4758000e-03,  3.6373200e-02,
       -5.1422000e-03, -1.1222900e-02, -3.6812000e-03,  5.6755100e-02])

In [233]:
TimeA - Tarr_A.flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [90]:
reg_new = np.polyfit(Time, Dist_eq,1)
reg_old = np.polyfit(Tarr_T[~np.isnan(Tarr_T)], Dist_eq[~np.isnan(Tarr_T)],1)

In [91]:
reg_new[0]

1513.3652157403837

In [92]:
reg_old[0]

1511.3067996970028

In [247]:
## SAME
Time - Tarr_T.flatten()


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [253]:
## avec la bonne densité (et 12000 NBeams) : la seule différence est dans les angles limites....
Time - Tarr_T.flatten()

array([ 5.1922800e-03,  1.8030000e-04,  1.5502900e-02, -4.3110000e-04,
       -2.2541000e-02, -1.3085000e-03, -6.7406000e-03, -2.7439100e-02,
       -4.2526300e-02,  6.5412600e-03,  4.7700000e-05, -4.1960000e-04,
        7.2670000e-04,  7.4195900e-02, -2.0103000e-03, -7.6675000e-03,
       -2.9064200e-02, -4.2926800e-02,  1.9955700e-03, -7.8200000e-05,
       -3.8000000e-06,  1.7700000e-03,  6.7256900e-02, -3.1166000e-03,
        1.4801000e-03, -2.8636900e-02,  1.0053215e+00,  1.4982200e-03,
       -1.9270000e-04,  3.6300000e-05, -2.1833400e-02,  5.9581800e-02,
       -2.1591000e-03, -8.5068000e-03, -2.4417900e-02,            nan])

In [ ]:
### différence de temps d'arrivées déjà très importante .... Plusieurs mètres d'erreur.

In [178]:
## avec la bonne densité (mais 60000 NBeams)
Time - Tarr_T.flatten()

array([-3.809450e-03, -9.820000e-05,  1.830300e-02,  6.619000e-04,
        3.147100e-03,  4.241900e-03,  1.630780e-02,  3.515240e-02,
       -1.940920e-02,  2.836220e-03,  2.231000e-04, -1.729770e-02,
        7.476000e-04, -1.206590e-02,  2.052300e-02,  1.349260e-02,
        4.898070e-02, -6.291657e-01, -3.643100e-04,  5.570000e-04,
       -1.667790e-02,  3.910000e-04, -4.292680e-02,  3.041460e-02,
       -4.692000e-04,  2.893450e-02, -4.089350e-02, -5.078300e-04,
       -2.318000e-04, -2.300300e-03, -1.264600e-03, -8.396200e-03,
       -6.180000e-04,  2.048500e-02, -3.074270e-02,  3.002200e-03])

In [171]:
## avec 12000Beams (au lieu de 60000 dans OLD)
Time - Tarr_T.flatten()

array([ 4.599100e-03,  1.879000e-04,  1.709750e-02,  9.594000e-04,
        5.684000e-04, -9.041000e-04,  9.750400e-03,  4.818000e-03,
       -4.267890e-02,  9.330750e-03,  2.250000e-04,  1.055530e-02,
        1.298900e-03,  6.503300e-02, -4.578000e-04,  1.080710e-02,
        2.375259e-01, -4.336930e-02,  3.229620e-03,  1.133900e-03,
        3.776000e-04,  1.634600e-03,  3.059010e-02,  1.789100e-03,
        1.200860e-02,  3.734970e-02, -1.716610e-02,  1.451970e-03,
        1.196800e-03,  1.077700e-03, -2.430920e-02,  2.320860e-02,
        2.983100e-03,  2.716100e-03, -2.073670e-02, -5.203743e-01])

In [160]:
# 51*101 receivers et autres paramètres différents
DIST = Dist_eq[10:-10:10, 10:-10:10].flatten()
Tarr_TT = Tarr_T[10:-10:10, 10:-10:10].flatten()
Time - Tarr_TT

array([-3.809450e-03, -9.820000e-05,  1.830300e-02,  6.619000e-04,
        3.147100e-03,  4.241900e-03,  1.630780e-02,  3.515240e-02,
       -1.940920e-02,  2.836220e-03,  2.231000e-04, -1.729770e-02,
        7.476000e-04, -1.206590e-02,  2.052300e-02,  1.349260e-02,
        5.788420e-02, -5.899048e-01, -3.643100e-04,  5.570000e-04,
       -1.667790e-02,  3.910000e-04, -4.292680e-02,  3.041460e-02,
       -4.692000e-04,  2.893450e-02, -4.089350e-02, -5.078300e-04,
       -2.318000e-04, -2.300300e-03, -1.264600e-03, -8.396200e-03,
       -6.180000e-04,  2.048500e-02, -3.074270e-02,  3.002200e-03])

In [96]:
# même nombre de receivers (6*11) mais autres paramètres différents
Time-Tarr_T

array([-3.809450e-03, -9.820000e-05,  1.830300e-02,  6.619000e-04,
        3.147100e-03,  4.241900e-03,  1.630780e-02,  3.515240e-02,
       -1.940920e-02,  2.836220e-03,  2.231000e-04, -1.729770e-02,
        7.476000e-04, -1.206590e-02,  2.052300e-02,  1.349260e-02,
        5.788420e-02, -5.899048e-01, -3.643100e-04,  5.570000e-04,
       -1.667790e-02,  3.910000e-04, -4.292680e-02,  3.041460e-02,
       -4.692000e-04,  2.893450e-02, -4.089350e-02, -5.078300e-04,
       -2.318000e-04, -2.300300e-03, -1.264600e-03, -8.396200e-03,
       -6.180000e-04,  2.048500e-02, -3.074270e-02,  3.002200e-03])

In [80]:
T_old = Tarr_T
T_old

array([ 6.62892723, 13.2443171 , 19.8231735 , 26.755064  , 33.7304077 ,
       40.1597977 , 46.1760368 , 54.7924232 , 61.8389893 ,  6.61886597,
       13.2362165 , 19.8095551 , 26.7407227 , 33.0873375 , 39.7145424 ,
       46.2122421 , 54.8614197 , 60.0436821 ,  6.62893772, 13.2513056 ,
       19.8002396 , 26.3880501 , 33.2838974 , 40.1133537 , 47.179863  ,
       53.1380348 ,         nan,  6.62180281, 13.2231321 , 19.8433666 ,
       26.4626808 , 33.2971344 , 40.0945969 , 46.1453781 , 53.6137924 ,
       60.5702972 ])

In [98]:
Time - T_old

array([-0.00414514, -0.0530453 , -0.0259704 , -0.3805332 , -0.6804352 ,
       -0.5317345 ,  0.0068131 , -1.9163589 , -2.41428   ,  0.00283622,
       -0.0499506 , -0.01787   , -0.372202  , -0.0677414 , -0.0949326 ,
       -0.039566  , -1.7610512 , -0.6291695 , -0.01651907, -0.0703497 ,
       -0.017706  , -0.0255413 , -0.2952461 , -0.5003967 , -1.0188523 ,
       -0.2462425 ,         nan, -0.01443672, -0.0483389 , -0.0591068 ,
       -0.1061821 , -0.3123665 , -0.4902268 ,  0.0050202 , -0.7884903 ,
       -1.1435928 ])

In [ ]:
### La différence vient des fichiers .arr et donc des fichiers d'entrées (.env). 

# Le profil est exactement le même.
# La densité du fond change
# Le nombre de récepteurs change :
#       med1_OLd (avec 6*11 receivers) & med1_51_101.arr (avec 51*101 receivers) : Mêmes temps d'arrivée

# le nombre d'angles NBeams change
# Les angles limites ALimits changent


### Il faut faire des tests pour déterminer le paramètre qui change les temps d'arrivées ... 